# Generative AI playground
Quick start: pip install -q -U google-genai


## import stuff

In [3]:
import config
from google import genai
from google.genai import types
from IPython.display import Markdown

# import google.generativeai as genai

# Rest of your code remains the same



## Connect to gemini
only need to do this once

In [4]:
client = genai.Client(api_key=config.apikey)

## Code base: basic text generation

In [ ]:

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Discuss differences and similarity between Pikachu and Lord Voldermort in less than 100 words",
)

display(Markdown(response.text))

## Image input

We will use Pillow library to deal with image data.

In [ ]:
from PIL import Image
from IPython.display import display

### local image input

In [ ]:
image = Image.open('./img1.png')

# Display the image inline in the notebook
display(image)


In [ ]:
client = genai.Client(api_key=config.apikey)
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["What is this image?", image])

Display(mresponse.text)

### Public image

In [ ]:
from io import BytesIO
import requests

image_path = r"https://iheartcraftythings.com/wp-content/uploads/2021/04/Harry-Potter-DRAWING-%E2%80%93-STEP-10.jpg"
response = requests.get(image_path)  # Changed variable name from 'image' to 'response'

# Open image from the response
image2 = Image.open(BytesIO(response.content))

# Display the image
display(image2)

In [ ]:
client = genai.Client(api_key=config.apikey)
response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents=["What is this image?", image2])

print(response.text)

## Audio 

In [ ]:
from IPython.display import Audio

# Replace 'your_file.mp3' with the actual path to your MP3 file
Audio("./noise.mp3", autoplay=True)


In [ ]:
import os

# myfile = client.files.upload(path='noise.mp3')

with open('noise.mp3', 'rb') as f:
    image_bytes = f.read()

response = client.models.generate_content(
  model='gemini-2.0-flash',
  contents=[
    'Describe this audio clip',
    types.Part.from_bytes(
      data=image_bytes,
      mime_type='audio/mp3',
    )
  ]
)

print(response.text)

In [ ]:

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Those 2 characters are from different franchises. One from Pokemon, the other from Harry potter. Do you remember what I asked you?.",
)

print(response.text)

## PDF file input

### Use online public pdf file

In [ ]:
import httpx

doc_url = "https://discovery.ucl.ac.uk/id/eprint/10089234/1/343019_3_art_0_py4t4l_convrt.pdf"  # Replace with the actual URL of your PDF

# Retrieve and encode the PDF byte
doc_data = httpx.get(doc_url).content

prompt = "Summarize this document"
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
      types.Part.from_bytes(
        data=doc_data,
        mime_type='application/pdf',
      ),
      prompt])
display(Markdown(response.text))

### Use local pdf file

In [ ]:
import pathlib

# Retrieve and encode the PDF byte
filename = "chap2.pdf"  # Replace with your local PDF filename

# Read the local PDF file as bytes
with open(filename, "rb") as f:
    doc_data = f.read()


prompt = "what is this document all about?"
response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[
        types.Part.from_bytes(
            data=doc_data,
            mime_type="application/pdf",
        ),
      prompt])
display(Markdown(response.text))

### Structured output

In a lot of cases, one might want output to be of a certain structure, like json. Here is how:

### Just do it in the prompt

In [5]:

prompt = """Generate a Harry Potter quiz. 10 Questions. Based on only Book1 to Book4 of the series.

Use this JSON schema:

questions = {'question': str, 'answer': str}
Return: list[questions]"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt,
)

# Use the response as a JSON string.
# print(response.text)

In [7]:
import json
import pandas as pd

raw_string = response.text
json_string = raw_string.strip("```json\n")

# Parse the JSON string
data = json.loads(json_string)

pd.DataFrame(data)


,question,answer
0,What is the name of Harry Potter's pet owl?,Hedwig
1,In which house does the Sorting Hat initially ...,Slytherin
2,What position does Harry play on the Gryffindo...,Seeker
3,What is the name of the three-headed dog guard...,Fluffy
4,What is the name of Ron Weasley's pet rat?,Scabbers
5,What position does Gilderoy Lockhart teach at ...,Defence Against the Dark Arts
6,What creature is Hagrid raising in his hut in ...,Buckbeak (a Hippogriff)
7,Who is the real traitor who betrayed the Potte...,Peter Pettigrew
8,What is the first task in the Triwizard Tourna...,Retrieving a golden egg from a dragon
9,Who is the champion selected from Durmstrang f...,Viktor Krum


In [8]:
data

[{'question': "What is the name of Harry Potter's pet owl?",
  'answer': 'Hedwig'},
 {'question': 'In which house does the Sorting Hat initially consider placing Harry?',
  'answer': 'Slytherin'},
 {'question': 'What position does Harry play on the Gryffindor Quidditch team?',
  'answer': 'Seeker'},
 {'question': "What is the name of the three-headed dog guarding the trapdoor in Philosopher's Stone?",
  'answer': 'Fluffy'},
 {'question': "What is the name of Ron Weasley's pet rat?",
  'answer': 'Scabbers'},
 {'question': 'What position does Gilderoy Lockhart teach at Hogwarts in Chamber of Secrets?',
  'answer': 'Defence Against the Dark Arts'},
 {'question': 'What creature is Hagrid raising in his hut in Prisoner of Azkaban which gets him into trouble?',
  'answer': 'Buckbeak (a Hippogriff)'},
 {'question': 'Who is the real traitor who betrayed the Potters to Voldemort?',
  'answer': 'Peter Pettigrew'},
 {'question': 'What is the first task in the Triwizard Tournament in Goblet of Fir